In [17]:
import pandas as pd
import numpy as np 
#from faker import Faker
import re, json, logging , html, os
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_parser import issue_parser
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.dbupdate_parser import dbupdate_parser
from lib.script_generator import script_generator 
from lib.db_parser import db_parser   
from bs4 import BeautifulSoup    
#from nltk.corpus import stopwords 
#sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore') 


In [45]:
file=f'{ctx.get_down_dir()}ig.xlsx' 
converts={'Question':lambda x: re.sub('\n','',x) } 
dff=pd.read_excel(file, converters=converts, sheet_name=int(0), header=0 ) 
df=dff.copy()
df=df.dropna(axis=1,how='all')
df.to_html('out/script.html')
 




    DROP TABLE @T;
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE NAME = '@T' AND TYPE = 'U')
    BEGIN
        CREATE TABLE [dbo].@T  (
        [PK_@T] [INT] IDENTITY(1,1) NOT NULL
        , Question NVARCHAR(4000) NULL
		, Unnamed: 4 NVARCHAR(4000) NULL
		, Unnamed: 5 NVARCHAR(4000) NULL
		, Criteria NVARCHAR(4000) NULL
		, Unnamed: 7 NVARCHAR(4000) NULL
		, Unnamed: 11 NVARCHAR(4000) NULL
		, Review Cycle NVARCHAR(4000) NULL
		, Unnamed: 13 NVARCHAR(4000) NULL
		, Unnamed: 14 NVARCHAR(4000) NULL
		, Unnamed: 15 NVARCHAR(4000) NULL
		, Maturity Level NVARCHAR(4000) NULL
		, Unnamed: 17 NVARCHAR(4000) NULL
		, Unnamed: 20 NVARCHAR(4000) NULL
		, Unnamed: 21 NVARCHAR(4000) NULL
		, Unnamed: 22 NVARCHAR(4000) NULL
		, Unnamed: 23 NVARCHAR(4000) NULL
		, Unnamed: 24 NVARCHAR(4000) NULL
		, Unnamed: 27 NVARCHAR(4000) NULL
		, Unnamed: 28 NVARCHAR(4000) NULL
		, Unnamed: 29 NVARCHAR(4000) NULL
		, Unnamed: 30 NVARCHAR(4000) NULL
		, Unnamed: 31 NVARCHAR(4000) NULL
		, Unnamed: 34 NVARC

In [2]:
df=issue_provider(ctx).provide( '8787', usecache=False )



====== WebDriver manager ======
Current google-chrome version is 114.0.5735
Get LATEST driver version for 114.0.5735
Driver [C:\Users\timko\.wdm\drivers\chromedriver\win32\114.0.5735.90\chromedriver.exe] found in cache


In [ ]:
issue_parser(ctx).parse('8787:8790')

# PIPELINE

#### requirements > jira > sql > forms
#### requirements/jira > sql > forms

# forms from sql

In [ ]:
parser = db_parser(ctx)
df=parser.parse(question_group='4406')
df

In [ ]:
code=script_generator(ctx).generate(df) 
print(code)

### Jira Parse Entire Datacall

In [ ]:
ip = issue_parser(ctx)     
df=ip.parse('9600:9605' )  
df

### sql from jira

In [ ]:
gen=script_generator(ctx)
code=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}aspx\\ig.sql')
print(code)

### NLP

In [ ]:
string = 'Effective / Not Effective'
picklist = string.split(' / ')
print ( picklist ) 
pr = picklist_recommender(ctx, use_cache=False) 
pl=pr.recommend(picklist, generate_sql=False)
pl

In [ ]:
class aspx_generator():
    def __init__(self, ctx):  
        self.ctx=ctx 
    def generate(self, PK_FORM='2023-A-HVA', classname=None, where=None, commit=False, PageNameFilter='~', dest=None ):
        parser=db_parser(ctx)
        generator=script_generator(ctx)
        sql= f"""
            SELECT DISTINCT PK_FORM, PK_QuestionGroup, SectionNum, GroupName, ASPX
            FROM vwQuestions  WHERE 1=1 AND PK_FORM='{PK_FORM}' 
            --AND {where}
            ORDER BY PK_QuestionGroup ASC
        """
        if classname == None: classname=PK_FORM.replace('-','_')
        if where != None: sql=sql.replace('--AND', 'AND  ')
        #print(sql)
        df=sql_todf(sql, self.ctx.connstr)     
        df['PNAME']=df['ASPX'].apply(lambda s: s.split('/')[len(s.split('/'))-1].replace('.aspx',''))
        pages=df.to_dict(orient='records')    
        src=ctx.get_tempalte_dir()
        out=''
        for p in pages:   
            PageName = re.sub(PageNameFilter,'',f"{classname}_{p['GroupName']}")
            for ext in ['.aspx','.aspx.designer.vb','.aspx.vb']:
                with open(f'{src}{classname}{ext}', 'r', encoding='UTF-8', errors='ignore') as fr: 
                    txt = fr.read().encode(  'utf-8', errors='ignore' ).decode('utf-8')
                    txt=txt.replace('{classname}',PageName) 
                    txt=txt.replace('{PK_key}',f"{p['PK_QuestionGroup']}")
                    txt=txt.replace('{GroupName}',f"{p['GroupName']}")  
                    if ext=='.aspx': 
                        parsed=parser.parse(question_group=p['PK_QuestionGroup'])
                        code = generator.generate(parsed)
                        txt=txt.replace(f'<!--controls-->',code) 
                    out=out+txt
                path=f"{dest}{p['PNAME']}{ext}"
                print(path)
                if commit:  
                    with open(path, 'w', encoding='UTF-8') as fw:
                        fw.write(txt)
        with open(fr"{ctx.get_dest()}\script.aspx", 'w', encoding='UTF-8') as fw: 
            fw.write(out)
 
dest=rf"C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\FismaForms\\2023\\"  
generate_pages(PK_FORM='2023-A-IG', where=' PK_QuestionGroup < 4461 ', commit=True, PageNameFilter='Function',dest=dest)

In [ ]:
txt = '8433'
with open ('cache/8433.html','r') as f:
    txt=f.read()
    txt=re.sub('&nbsp;', ' ', txt) 
    txt=re.sub('<p>\s*</p>', ' ', txt)  
    txt=re.sub('\n\s*\n', '\n', txt)
    txt=re.sub('\n{1,}', '\n', txt)
    
print(txt)
lines = re.split('(?:(\n?.*>?\d[\d\.a-z]+))\s*', txt )
metrics = zip(lines[1::2], lines[2::2])
metrics = [f'{x} {y}' for x,y  in list(metrics)]
metrics



In [ ]:
sql="""
    UPDATE CVE SET CDMCountSubmitted=[count], SubmissionCount=[count], CDMCountImported=[count]
    FROM CVEUnremediated CVE
    INNER JOIN vwOrgSubToComponent O ON O.PK_OrgSubmission=CVE.PK_OrgSubmission
    WHERE PK_ReportingCycle=108 And O.enableZeroTouch=1 And OrgSub_Description='ACTIVE'
    AND CVE.PK_CISA_CVE='[cve id]' And O.Acronym='PEACE'
"""

df = pd.read_csv(f'C:\\Users\\timko\\Downloads\\kev_update.csv')  
df=df.loc[df['agency'] =='PC']
df['sql']=[sql.replace('[agency]', str(x)).replace('[cve id]', str(y)).replace('[count]', str(z))   for x,y,z in zip(df['agency'], df['cve id'], df['count']) ]
update = df['sql'].str.cat() 
#print(update) 
with open(f'out/script.sql', 'w') as f:
    f.write(update)

,{idt},{QT_CODE},{FK_QuestionType},{PK_Question},{PK_PickListType},{sortpos},{QuestionText},{QT_ABBR},{PK_QuestionGroup},{Dependancy}
0,0_1,PICK,17,53000,528,1,Please provide an overall IG self-assessment r...,Please provide an overall IG self-assessment r...,4451,
1,1,PICK,17,53010,85,1,To what extent does the organization maintain ...,To what extent does the organization maintain ...,4452,
2,12,PICK,17,53031,85,1,To what extent does the organization use an or...,To what extent does the organization use an or...,4453,
3,17,PICK,17,53047,85,1,To what extent have the roles and responsibili...,To what extent have the roles and responsibili...,4454,
4,26,PICK,17,53066,85,1,To what extent have the roles and responsibili...,To what extent have the roles and responsibili...,4455,
...,...,...,...,...,...,...,...,...,...,...
66,34_1,PICK,17,53075,85,10,Please provide the assessed maturity level fo...,Please provide the assessed maturity level fo...,4455,
67,46_3,TXT,9,53110,85,10,Provide any additional information on the effe...,Provide any additional information on the effe...,4457,
68,34_2,TXT,9,53076,85,11,Provide any additional information on the eff...,Provide any additional information on the eff...,4455,
69,11_1,PICK,17,53020,85,12,Please provide the assessed maturity level for...,Please provide the assessed maturity level for...,4452,
